In [67]:
%pip install openpyxl lxml

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [68]:
import pandas as pd

In [69]:
hdi_df = pd.read_csv('./data/Human Development Index - Full.csv')

hdi_subset = hdi_df[['ISO3', 'Country', 'Human Development Index (2021)']].rename(columns={'Human Development Index (2021)': 'hdi_2021', 'Country': 'country'})

hdi_subset.to_json("./data/hdi.json", orient="index")

In [70]:
hdi_subset

,ISO3,country,hdi_2021
0,AFG,Afghanistan,0.478
1,AGO,Angola,0.586
2,ALB,Albania,0.796
3,AND,Andorra,0.858
4,ARE,United Arab Emirates,0.911
...,...,...,...
190,WSM,Samoa,0.707
191,YEM,Yemen,0.455
192,ZAF,South Africa,0.713
193,ZMB,Zambia,0.565


In [71]:
cost_of_living_df = pd.read_html('https://www.numbeo.com/cost-of-living/rankings_by_country.jsp')[1]

In [72]:
cost_of_living_df = cost_of_living_df.dropna(axis=1, how='all')

cost_of_living_df

,Country,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,Bermuda,141.8,96.3,120.0,143.3,142.0,79.5
1,Switzerland,114.2,50.2,83.5,113.9,109.5,118.7
2,Cayman Islands,103.4,75.7,90.1,96.4,92.4,76.7
3,Bahamas,90.9,39.6,66.3,74.1,94.8,43.2
4,Barbados,88.8,21.2,56.4,80.1,82.7,36.8
...,...,...,...,...,...,...,...
135,Libya,24.2,5.2,15.1,23.1,19.3,32.1
136,Colombia,23.1,7.0,15.4,21.0,17.3,31.1
137,India,22.4,5.5,14.3,23.3,15.7,64.6
138,Egypt,21.6,4.3,13.3,19.6,18.0,21.9


In [73]:
trimmed_col_df = cost_of_living_df[['Country', 'Cost of Living Index', 'Rent Index', 'Groceries Index']]

In [74]:
trimmed_col_df = trimmed_col_df.rename(columns={ 'Country': 'country', 'Cost of Living Index': 'cost_of_living', 'Rent Index': 'rent_index', 'Groceries Index': 'groceries_index'})

In [75]:
# Normalization
trimmed_col_df['cost_of_living'] /= trimmed_col_df['cost_of_living'].max()
trimmed_col_df['rent_index'] /= trimmed_col_df['rent_index'].max()
trimmed_col_df['groceries_index'] /= trimmed_col_df['groceries_index'].max()

In [76]:
trimmed_col_df

,country,cost_of_living,rent_index,groceries_index
0,Bermuda,1.000000,1.000000,1.000000
1,Switzerland,0.805360,0.521288,0.794836
2,Cayman Islands,0.729196,0.786085,0.672715
3,Bahamas,0.641044,0.411215,0.517097
4,Barbados,0.626234,0.220145,0.558967
...,...,...,...,...
135,Libya,0.170663,0.053998,0.161200
136,Colombia,0.162906,0.072690,0.146546
137,India,0.157969,0.057113,0.162596
138,Egypt,0.152327,0.044652,0.136776


In [118]:
migration_df = pd.read_excel('./data/destination_migrant_amount.xlsx', sheet_name=1).drop(['Unnamed: 0'], axis=1)
migration_df.columns.values[0] = 'country'
migration_df.columns.values[1] = 'immigration_2020'
migration_df['country'] = migration_df['country'].str.strip()
migration_df

,country,immigration_2020
0,Burundi,344767.0
1,Comoros,12496.0
2,Djibouti,119738.0
3,Eritrea,13934.0
4,Ethiopia,1085517.0
...,...,...
230,Samoa,4021.0
231,Tokelau,1238.0
232,Tonga,3742.0
233,Tuvalu,239.0


In [119]:
merged_df = pd.merge(hdi_subset, trimmed_col_df, on=["country"], how='inner')
merged_df = pd.merge(merged_df, migration_df, on='country', how='inner')
merged_df = merged_df.set_index('ISO3')

merged_df

,country,hdi_2021,cost_of_living,rent_index,groceries_index,immigration_2020
ISO3,,,,,,
ALB,Albania,0.796,0.263752,0.094496,0.218423,48810.0
ARE,United Arab Emirates,0.911,0.425247,0.353063,0.325192,8716332.0
ARG,Argentina,0.842,0.220028,0.071651,0.179344,2281728.0
ARM,Armenia,0.759,0.287729,0.244029,0.239358,190349.0
AUS,Australia,0.951,0.531030,0.404984,0.511514,7685860.0
...,...,...,...,...,...,...
USA,United States,0.921,0.510578,0.489097,0.498255,50632836.0
UZB,Uzbekistan,0.727,0.190409,0.111111,0.177251,1162007.0
YEM,Yemen,0.455,0.414669,0.082035,0.464759,387113.0


In [120]:

merged_df[merged_df['hdi_2021'].isna()]

,country,hdi_2021,cost_of_living,rent_index,groceries_index,immigration_2020
ISO3,,,,,,


In [121]:
merged_df = merged_df[merged_df.index.notnull()]

In [122]:
merged_df.to_json('./data/merged_data.json', orient='index')